# Custom Text Splitter with Metadata Extraction

In [1]:
from X_application_directory.src.snag_tokens import Usage

In [2]:
import re

# Pre-compile regex patterns
PAGE_NUM_PATTERN = re.compile(r"Page \d+ of \d+")
HYPHEN_WORD_PATTERN = re.compile(r"(\w+)-(\w+)")
PERIOD_COLON_PATTERN = re.compile(r"([.:])([^ \n])")
CLOSE_PAREN_PATTERN = re.compile(r"(\))(?=[^\s])")
OPEN_PAREN_PATTERN = re.compile(r"(?<=[^\s])(\()")
WHITESPACE_PATTERN = re.compile(r"[ \t]+")


def remove_page_numbers(text):
    return re.sub(r"Page \d+ of \d+", "", text)


def remove_multiple_newlines(page_md):
    page_md = re.sub(r"\n\s*\n", "\n\n", page_md)
    return page_md


def fix_page(text):
    """
    Fixes common text problems in the given text.

    Args:
        text (str): The text to fix.

    Returns:
        str: The fixed text.
    """
    watermarks = [
        "SAMPLE",
        "HO 00 03 10 00 Copyright, Insurance Services Office, Inc., 1999",
        "HOMEOWNERS 3 – SPECIAL FORM",
        "HOMEOWNERS",
        "HO 00 03 10 00",
        "Copyright, Insurance Services Office, Inc., 1999",
    ]

    for watermark in watermarks:
        text = text.replace(watermark, "")

    text = remove_page_numbers(text)

    text = re.sub(HYPHEN_WORD_PATTERN, r"\1\2", text)
    text = re.sub(PERIOD_COLON_PATTERN, r"\1 \2", text)
    text = re.sub(CLOSE_PAREN_PATTERN, r"\1 ", text)
    text = re.sub(OPEN_PAREN_PATTERN, r" \1", text)
    text = re.sub(WHITESPACE_PATTERN, " ", text).strip()

    return text

In [3]:
from llama_index import SimpleDirectoryReader

documents_ho3 = SimpleDirectoryReader(input_files=['./data//HO3_sample.pdf']).load_data()

In [4]:
len(documents_ho3)

22

In [5]:
print(documents_ho3[0].text)

HOMEOWNERS
HO 00 03 10 00
HO 00 03 10 00 Copyright, Insurance Services Office, Inc., 1999 Page 1 of 22HOMEOWNERS 3 – SPECIAL FORM
AGREEMENT
We will provide the insurance described in this policy
in return for the premium and compliance with allapplicable provisions of this policy.
DEFINITIONS
A.In this policy, "you" and "your" refer to the "named
insured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this in-surance.
B.In addition, certain words and phrases are definedas follows:
1."Aircraft Liability", "Hovercraft Liability", "Motor
Vehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the
following:
a.Liability for "bodily injury" or "property dam-age" arising out of the:
(1)Ownership of such vehicle or craft by an"insured";
(2)Maintenance, occupancy, operation,use, loading or unloading of such vehi-cle or craft by any person;
(3)Entrustment of such vehicle or craft b

In [6]:
# Apply cleaning function to each page
for i, _ in enumerate(documents_ho3):
    documents_ho3[i].text = fix_page(documents_ho3[i].text)

## Transform text so that:
* Chunks ignore page breaks, but respect sentence boundry
* We limit wasted space while normalizing chunk size (except for the last one)

In [7]:
# Transform the pdf into a singe string

long_string = "".join(documents_ho3[i].text for i in range(len(documents_ho3)))
print(long_string[:2000])

AGREEMENT
We will provide the insurance described in this policy
in return for the premium and compliance with allapplicable provisions of this policy.
DEFINITIONS
A. In this policy, "you" and "your" refer to the "named
insured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this insurance.
B. In addition, certain words and phrases are definedas follows:
1. "Aircraft Liability", "Hovercraft Liability", "Motor
Vehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the
following:
a. Liability for "bodily injury" or "property damage" arising out of the:
(1) Ownership of such vehicle or craft by an"insured";
(2) Maintenance, occupancy, operation,use, loading or unloading of such vehicle or craft by any person;
(3) Entrustment of such vehicle or craft byan "insured" to any person;
(4) Failure to supervise or negligent supervision of any person involving such vehicle or craft b

### Create top level document

In [8]:
from llama_index.schema import Document

ho3_document = [Document(text=long_string,
                        id_="HO3_sample.pdf",
                        metadata={"Document Name": "HO3 Sample Policy", 
                                  "Category": "Homeowner's Insurance"})]

In [9]:
print(ho3_document[0].get_content())

AGREEMENT
We will provide the insurance described in this policy
in return for the premium and compliance with allapplicable provisions of this policy.
DEFINITIONS
A. In this policy, "you" and "your" refer to the "named
insured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this insurance.
B. In addition, certain words and phrases are definedas follows:
1. "Aircraft Liability", "Hovercraft Liability", "Motor
Vehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the
following:
a. Liability for "bodily injury" or "property damage" arising out of the:
(1) Ownership of such vehicle or craft by an"insured";
(2) Maintenance, occupancy, operation,use, loading or unloading of such vehicle or craft by any person;
(3) Entrustment of such vehicle or craft byan "insured" to any person;
(4) Failure to supervise or negligent supervision of any person involving such vehicle or craft b

In [10]:
from llama_index.langchain_helpers.text_splitter import SentenceSplitter
from llama_index import ServiceContext, set_global_service_context
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding
from llama_index.node_parser import SimpleNodeParser

text_splitter = SentenceSplitter(
  chunk_size=512,
  chunk_overlap=0,
)

node_parser = SimpleNodeParser(
    text_splitter=text_splitter,
)

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
embed_model = OpenAIEmbedding()

service_context = ServiceContext.from_defaults(
    llm=llm,
    node_parser=node_parser,
    embed_model=embed_model
    )


In [11]:
usage = Usage()

In [12]:
# Parse the doc
ho3_nodes = node_parser.get_nodes_from_documents(ho3_document)

In [13]:
print(usage.get())

[]


In [14]:
usage.total_tokens()

0

In [15]:
len(ho3_nodes)

38

In [16]:
ho3_nodes[0]

TextNode(id_='1bd09902-1f3f-433f-bb48-7d8ca75a9519', embedding=None, metadata={'Document Name': 'HO3 Sample Policy', 'Category': "Homeowner's Insurance"}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='HO3_sample.pdf', node_type=None, metadata={'Document Name': 'HO3 Sample Policy', 'Category': "Homeowner's Insurance"}, hash='624273b341f24db10da48b854351ea5645ddd6e5fc71c5f4ba9c2faf8fb33a3b'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8bcb35a3-94ea-4346-84a0-3ceae62dceda', node_type=None, metadata={'Document Name': 'HO3 Sample Policy', 'Category': "Homeowner's Insurance"}, hash='0eb5badc2e0a307322938347531fd6ad5274dc662a3876b78ac8fb1016f9bf59')}, hash='a73b8c34016dac3abb192a2a48876123f110ac0b67db90b7ce386332e2864db0', text='AGREEMENT\nWe will provide the insurance described in this policy\nin return for the premium and compliance with allapplicable provisions of this policy.\nDEFINITIONS

## The following code adds page numbers back to each node
* Chunks can span multiple pages

In [17]:
def identify_original_documents(documents, chunks):
    """
    This function identifies the original document(s) each chunk came from.
    Args:
    documents: A list of original documents.
    chunks: A list of text chunks.
    Returns:
    chunks: The provided chunks with appended page info.
    """
    # Combine all documents into a single string and track their ranges.
    long_string = ""
    pages_dict = {}
    for i, doc in enumerate(documents):
        start = len(long_string)
        long_string += doc.text
        end = len(long_string)
        pages_dict[i] = (start, end)

    # Identify the original document(s) for each chunk.
    for i, chunk in enumerate(chunks):
        chunk_start = long_string.find(chunk.text)
        chunk_end = chunk_start + len(chunk.text)
        pages = []
        for doc_number, (doc_start, doc_end) in pages_dict.items():
            if (chunk_start >= doc_start and chunk_start < doc_end) or (chunk_end > doc_start and chunk_end <= doc_end):
                pages.append(doc_number + 1)
        # Update existing metadata dictionary
        chunk.metadata.update({'pages': '-'.join(map(str, pages))})

    return chunks


In [18]:
# Running the function
nodes_with_pages = identify_original_documents(documents_ho3, ho3_nodes)

In [19]:
new_nodes = []

for node in nodes_with_pages:
    node.id_ = f"{node.metadata['Document Name']} | Pages: {node.metadata['pages']}"
    new_nodes.append(node)

In [20]:
new_nodes[0]

TextNode(id_='HO3 Sample Policy | Pages: 1', embedding=None, metadata={'Document Name': 'HO3 Sample Policy', 'Category': "Homeowner's Insurance", 'pages': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='HO3_sample.pdf', node_type=None, metadata={'Document Name': 'HO3 Sample Policy', 'Category': "Homeowner's Insurance"}, hash='624273b341f24db10da48b854351ea5645ddd6e5fc71c5f4ba9c2faf8fb33a3b'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8bcb35a3-94ea-4346-84a0-3ceae62dceda', node_type=None, metadata={'Document Name': 'HO3 Sample Policy', 'Category': "Homeowner's Insurance"}, hash='0eb5badc2e0a307322938347531fd6ad5274dc662a3876b78ac8fb1016f9bf59')}, hash='a73b8c34016dac3abb192a2a48876123f110ac0b67db90b7ce386332e2864db0', text='AGREEMENT\nWe will provide the insurance described in this policy\nin return for the premium and compliance with allapplicable provisions of this policy.\nDEFIN

In [21]:
from llama_index.schema import MetadataMode

print(
    "LLM sees:\n",
    new_nodes[0].get_content(metadata_mode=MetadataMode.LLM),
)

LLM sees:
 Document Name: HO3 Sample Policy
Category: Homeowner's Insurance
pages: 1

AGREEMENT
We will provide the insurance described in this policy
in return for the premium and compliance with allapplicable provisions of this policy.
DEFINITIONS
A. In this policy, "you" and "your" refer to the "named
insured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this insurance.
B. In addition, certain words and phrases are definedas follows:
1. "Aircraft Liability", "Hovercraft Liability", "Motor
Vehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the
following:
a. Liability for "bodily injury" or "property damage" arising out of the:
(1) Ownership of such vehicle or craft by an"insured";
(2) Maintenance, occupancy, operation,use, loading or unloading of such vehicle or craft by any person;
(3) Entrustment of such vehicle or craft byan "insured" to any person;
(4) Failure

In [22]:
print(
    "Embeddings model sees:\n",
    new_nodes[0].get_content(metadata_mode=MetadataMode.EMBED)),

Embeddings model sees:
 Document Name: HO3 Sample Policy
Category: Homeowner's Insurance
pages: 1

AGREEMENT
We will provide the insurance described in this policy
in return for the premium and compliance with allapplicable provisions of this policy.
DEFINITIONS
A. In this policy, "you" and "your" refer to the "named
insured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this insurance.
B. In addition, certain words and phrases are definedas follows:
1. "Aircraft Liability", "Hovercraft Liability", "Motor
Vehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the
following:
a. Liability for "bodily injury" or "property damage" arising out of the:
(1) Ownership of such vehicle or craft by an"insured";
(2) Maintenance, occupancy, operation,use, loading or unloading of such vehicle or craft by any person;
(3) Entrustment of such vehicle or craft byan "insured" to any person

(None,)

In [23]:
print(new_nodes[1].get_metadata_str())

Document Name: HO3 Sample Policy
Category: Homeowner's Insurance
pages: 1-2


In [59]:
from llama_index.schema import TextNode

test = new_nodes[0].to_langchain_format

AttributeError: 'TextNode' object has no attribute 'to_langchain_format'

In [24]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex(new_nodes, service_context=service_context)
engine = index.as_query_engine(similarity_top_k=10)


In [25]:
from llama_index.indices.postprocessor import LLMRerank

reranker = LLMRerank(choice_batch_size=10, top_n=4)

In [26]:
rerank_engine = index.as_query_engine(
    node_postprocessors=[reranker], service_context=service_context)

In [27]:
from llama_index.question_gen.llm_generators import LLMQuestionGenerator
from llama_index.question_gen.prompts import DEFAULT_SUB_QUESTION_PROMPT_TMPL

question_gen = LLMQuestionGenerator.from_defaults(
    prompt_template_str="""
        Follow the example, but instead of giving a question, always prefix the question 
        with: 'By first identifying and quoting the most relevant sources, '. 
        """
    + DEFAULT_SUB_QUESTION_PROMPT_TMPL,
)

In [28]:
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata


final_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine,
            metadata=ToolMetadata(
                name="homeowners_policy_documents",
                description="insurance policy contract detailing coverage provisions.",
            ),
        )
    ],
    question_gen=question_gen,
    service_context=service_context,
    use_async=False,
)

In [29]:
response = final_engine.query(
    """
    How does the policy addresses water damage for dwelling damage and personal property damage?
    """
)
print(response.response)

Generated 2 sub questions.
[homeowners_policy_documents] Q: By first identifying and quoting the most relevant sources, what does the homeowners policy document say about water damage for dwelling damage?
[homeowners_policy_documents] A: According to the HO3 Sample Policy document, under Section I - Exclusion A. 3. Water Damage, it states that "Paragraphs a. and c. that apply to surface water and water below the surface of the ground do not apply to loss by water covered under c. (5) and (6) above." This means that water damage resulting from an accidental discharge or overflow of water or steam from within a storm drain, water, steam or sewer pipe, plumbing, heating, air conditioning or automatic fire protective sprinkler system, or household appliance on the "residence premises" is covered, unless otherwise excluded.
[homeowners_policy_documents] Q: By first identifying and quoting the most relevant sources, what does the homeowners policy document say about water damage for personal

In [30]:
from IPython.display import Markdown

Markdown(f"{response}")

The policy addresses water damage for dwelling damage by stating that water damage resulting from an accidental discharge or overflow of water or steam from specified systems or appliances is covered, unless otherwise excluded. This means that water damage caused by these incidents is covered under the policy.

For personal property damage, the policy also covers water damage under certain conditions. It states that water damage caused by an accidental discharge or overflow of water or steam from specified systems or appliances is covered. However, it is important to note that the policy does not cover the repair or replacement of the system or appliance from which the water or steam escaped.

In [31]:
Markdown(f"{response.source_nodes[0].node.text}")

Sub question: By first identifying and quoting the most relevant sources, what does the homeowners policy document say about water damage for dwelling damage?
Response: According to the HO3 Sample Policy document, under Section I - Exclusion A. 3. Water Damage, it states that "Paragraphs a. and c. that apply to surface water and water below the surface of the ground do not apply to loss by water covered under c. (5) and (6) above." This means that water damage resulting from an accidental discharge or overflow of water or steam from within a storm drain, water, steam or sewer pipe, plumbing, heating, air conditioning or automatic fire protective sprinkler system, or household appliance on the "residence premises" is covered, unless otherwise excluded.

In [32]:
Markdown(f"{response.source_nodes[1].node.text}")

Sub question: By first identifying and quoting the most relevant sources, what does the homeowners policy document say about water damage for personal property damage?
Response: According to the HO3 Sample Policy document, water damage to personal property is covered under certain conditions. Specifically, under Section I - Exclusion A. 3. Water Damage, it states that paragraphs a. and c. that apply to surface water and water below the surface of the ground do not apply to loss by water covered under c. (5) and (6) above. This means that water damage caused by an accidental discharge or overflow of water or steam from within certain specified systems or appliances is covered. However, it is important to note that the policy does not cover the repair or replacement of the system or appliance from which the water or steam escaped.

In [33]:
response = final_engine.query(
    """
    What are the differences in how the policy addresses water damage for dwelling damage and personal property damage?
    """
)
print(response.response)

Generated 2 sub questions.
[homeowners_policy_documents] Q: By first identifying and quoting the most relevant sources, what does the policy say about water damage for dwelling damage?
[homeowners_policy_documents] A: According to the policy, water damage to the dwelling is covered under certain circumstances. Specifically, it states that loss to property covered under Coverage A or B resulting from an accidental discharge or overflow of water or steam from within a storm drain, water, steam or sewer pipe, or plumbing, heating, air conditioning, or automatic fire protective sprinkler system or household appliance on the "residence premises" is covered. This coverage includes the cost to tear out and replace any part of a building or other structure on the "residence premises" necessary to repair the system or appliance. However, the policy does not cover loss to the system or appliance from which the water or steam escaped. This information is found on pages 7-8 of the HO3 Sample Polic

In [34]:
Markdown(f"{response.response}")

The policy addresses water damage for dwelling damage by covering loss to property resulting from an accidental discharge or overflow of water or steam from within certain systems or appliances on the "residence premises." It also covers the cost to tear out and replace any part of a building or structure necessary for repair. However, it does not cover loss to the system or appliance from which the water or steam escaped.

On the other hand, the policy addresses water damage for personal property damage by covering direct physical loss to the property described in Coverage C caused by certain perils, unless the loss is excluded. It specifically mentions that loss to property contained in a building caused by rain, snow, sleet, sand, or dust is not covered, unless the direct force of wind or hail damages the building causing an opening in a roof or wall and the water enters through this opening. This suggests that water damage to personal property is not covered unless it is caused by a covered peril such as wind or hail damaging the building.

In [35]:
Markdown(f"{response.source_nodes[0].node.text}")

Sub question: By first identifying and quoting the most relevant sources, what does the policy say about water damage for dwelling damage?
Response: According to the policy, water damage to the dwelling is covered under certain circumstances. Specifically, it states that loss to property covered under Coverage A or B resulting from an accidental discharge or overflow of water or steam from within a storm drain, water, steam or sewer pipe, or plumbing, heating, air conditioning, or automatic fire protective sprinkler system or household appliance on the "residence premises" is covered. This coverage includes the cost to tear out and replace any part of a building or other structure on the "residence premises" necessary to repair the system or appliance. However, the policy does not cover loss to the system or appliance from which the water or steam escaped. This information is found on pages 7-8 of the HO3 Sample Policy document.

In [36]:
Markdown(f"{response.source_nodes[1].node.text}")

Sub question: By first identifying and quoting the most relevant sources, what does the policy say about water damage for personal property damage?
Response: According to the HO3 Sample Policy, under Coverage C - Personal Property, the policy covers direct physical loss to the property described in Coverage C caused by any of the following perils unless the loss is excluded. One of the perils mentioned is "Fire Or Lightning." However, it specifically states that this peril does not include loss to the property contained in a building caused by rain, snow, sleet, sand, or dust unless the direct force of wind or hail damages the building causing an opening in a roof or wall and the rain, snow, sleet, sand, or dust enters through this opening. This suggests that water damage to personal property is not covered unless it is caused by a covered peril such as wind or hail damaging the building.

In [37]:
response = final_engine.query(
    """
    What are the considerations for determining if the policy does or does not cover damaged caused by collapse?
    """
)
print(response.response)

Generated 1 sub questions.
[homeowners_policy_documents] Q: By first identifying and quoting the most relevant sources, what are the considerations for determining if the policy does or does not cover damaged caused by collapse?
[homeowners_policy_documents] A: According to the HO3 Sample Policy, the considerations for determining if the policy covers damage caused by collapse are as follows:

1. The collapse must be caused by one or more of the Perils Insured Against named under Coverage C.
2. Decay or insect or vermin damage that is hidden from view is covered, unless the presence of such decay or damage is known to the insured prior to collapse.
3. The weight of contents, equipment, animals, people, or rain that collects on a roof can cause a collapse and would be covered.
4. The use of defective material or methods in construction, remodeling, or renovation can cause a collapse, but coverage is only provided if the collapse occurs during the course of the construction, remodeling, 

In [38]:
Markdown(f"{response.response}")

The considerations for determining if the policy covers damage caused by collapse are as follows:

1. The collapse must be caused by one or more of the Perils Insured Against named under Coverage C.
2. Decay or insect or vermin damage that is hidden from view is covered, unless the presence of such decay or damage is known to the insured prior to collapse.
3. The weight of contents, equipment, animals, people, or rain that collects on a roof can cause a collapse and would be covered.
4. The use of defective material or methods in construction, remodeling, or renovation can cause a collapse, but coverage is only provided if the collapse occurs during the course of the construction, remodeling, or renovation.
5. Loss to certain property, such as awnings, fences, patios, decks, swimming pools, etc., is not covered unless it is a direct result of the collapse of a building or any part of a building.

Therefore, the policy does cover damage caused by collapse if it meets the specified conditions.

In [39]:
Markdown(f"{response.source_nodes[0].node.text}")

Sub question: By first identifying and quoting the most relevant sources, what are the considerations for determining if the policy does or does not cover damaged caused by collapse?
Response: According to the HO3 Sample Policy, the considerations for determining if the policy covers damage caused by collapse are as follows:

1. The collapse must be caused by one or more of the Perils Insured Against named under Coverage C.
2. Decay or insect or vermin damage that is hidden from view is covered, unless the presence of such decay or damage is known to the insured prior to collapse.
3. The weight of contents, equipment, animals, people, or rain that collects on a roof can cause a collapse and would be covered.
4. The use of defective material or methods in construction, remodeling, or renovation can cause a collapse, but coverage is only provided if the collapse occurs during the course of the construction, remodeling, or renovation.
5. Loss to certain property, such as awnings, fences, patios, decks, swimming pools, etc., is not covered unless it is a direct result of the collapse of a building or any part of a building.

Therefore, the policy does cover damage caused by collapse if it meets the specified conditions.

In [40]:
from llama_index import Prompt

my_qa_template = Prompt(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question below. Include sources, i.e. TITLE, sub-title, A., 1., a, (1)., used to generate your answer. \n"
    "\n{query_str}\n"
)

CUSTOM_REFINE_PROMPT_TMPL = Prompt(
    "The original question is as follows: {query_str}\n"
    "We have provided an existing answer: {existing_answer}\n"
    "We have the opportunity to refine the existing answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better answer the question. "
    "If the context isn't useful, return the original answer. "
    "answer the question below. Include sources, i.e. TITLE, sub-title, A., 1., a, (1)., used to generate your answer. "
    
)

In [41]:
custom_query_engine = index.as_query_engine(
    refine_template=CUSTOM_REFINE_PROMPT_TMPL,
    text_qa_template=my_qa_template,
    service_context=service_context)

# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=custom_query_engine,
        metadata=ToolMetadata(
            name="homeowners_policy", 
            description="insurance policy contract detailing coverage provisions."
        ),
    )
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True,
)

In [42]:
response = await query_engine.aquery(
    "What are the considerations for determining if the policy does or does not cover damaged caused by collapse?"
)

Generated 2 sub questions.
[homeowners_policy] Q: Does the homeowners policy cover damage caused by collapse?
[homeowners_policy] Q: What are the coverage provisions in the homeowners policy?
[homeowners_policy] A: The coverage provisions in the homeowners policy include Coverage A - Dwelling, Coverage B - Other Structures, and Coverage C - Personal Property. 

Source: Document Name: HO3 Sample Policy, Category: Homeowner's Insurance, pages: 2-3
[homeowners_policy] A: Yes, the homeowners policy does cover damage caused by collapse. According to the HO3 Sample Policy, under Coverage C, the policy insures for direct physical loss to covered property involving collapse of a building or any part of a building if the collapse was caused by certain specified perils or factors such as decay, insect or vermin damage, weight of contents or rain, and use of defective material or methods in construction. (HO3 Sample Policy, Coverage C, pages 7-8)


In [43]:
Markdown(f"{response.response}")

The considerations for determining if the policy covers damage caused by collapse include checking the HO3 Sample Policy and specifically looking at Coverage C. The policy will cover direct physical loss to covered property involving collapse of a building or any part of a building if the collapse was caused by certain specified perils or factors such as decay, insect or vermin damage, weight of contents or rain, and use of defective material or methods in construction.

In [44]:
Markdown(f"{response.source_nodes[0].node.get_text()}")

Sub question: Does the homeowners policy cover damage caused by collapse?
Response: Yes, the homeowners policy does cover damage caused by collapse. According to the HO3 Sample Policy, under Coverage C, the policy insures for direct physical loss to covered property involving collapse of a building or any part of a building if the collapse was caused by certain specified perils or factors such as decay, insect or vermin damage, weight of contents or rain, and use of defective material or methods in construction. (HO3 Sample Policy, Coverage C, pages 7-8)

In [45]:
Markdown(f"{response.source_nodes[1].node.get_text()}")

Sub question: What are the coverage provisions in the homeowners policy?
Response: The coverage provisions in the homeowners policy include Coverage A - Dwelling, Coverage B - Other Structures, and Coverage C - Personal Property. 

Source: Document Name: HO3 Sample Policy, Category: Homeowner's Insurance, pages: 2-3

In [46]:
query = "Provide a detailed explanation of Personal Property coverage."

response = await query_engine.aquery(query)

Generated 4 sub questions.
[homeowners_policy] Q: What is Personal Property coverage?
[homeowners_policy] Q: What does Personal Property coverage include?
[homeowners_policy] Q: What are the limits of Personal Property coverage?
[homeowners_policy] Q: How does Personal Property coverage work?
[homeowners_policy] A: Personal Property coverage includes coverage for various types of property such as platinumplated ware, pewterware, property used primarily for business purposes, electronic apparatus and accessories, and other specified items such as money, securities, watercraft, trailers, jewelry, firearms, and silverware. 

Source: 
Document Name: HO3 Sample Policy
Category: Homeowner's Insurance
Page: 4
[homeowners_policy] A: Personal Property coverage refers to the insurance coverage provided for personal belongings or possessions owned by the insured individual. It includes coverage for items such as furniture, clothing, electronics, jewelry, and other personal items. The coverage lim

In [47]:
Markdown(f"{response}")

Personal Property coverage is a type of insurance coverage that protects the personal belongings or possessions owned by an individual. This coverage is typically included in homeowner's insurance policies. It provides compensation for the loss or damage of various types of personal property, such as furniture, clothing, electronics, jewelry, and other personal items.

The coverage limit for Personal Property is usually determined based on a percentage of the limit of liability for Coverage C or a specific dollar amount, whichever is greater. This means that the insurance policy will specify the maximum amount of compensation that can be claimed for personal property losses.

Personal Property coverage includes a wide range of items. Some examples of the types of property covered under this coverage include platinum-plated ware, pewterware, property used primarily for business purposes, electronic apparatus and accessories, money, securities, watercraft, trailers, jewelry, firearms, and silverware.

However, it is important to note that there may be special limits of liability for certain categories of personal property. These special limits do not increase the overall coverage limit for Personal Property. For example, there may be specific limits for money, securities, watercraft, trailers, jewelry, firearms, and silverware. These limits define the maximum amount of compensation that can be claimed for losses in these specific categories.

In the event of a loss or damage to personal property, the insured individual is required to follow certain procedures to make a claim. They may need to prepare an inventory of the damaged personal property, including details such as quantity, description, actual cash value, and amount of loss. They may also need to provide supporting documents such as bills, receipts, and related documents.

The insurance company may request the insured to show the damaged property, provide records and documents, and submit to an examination under oath. This is done to verify the validity of the claim and ensure that the compensation is appropriate.

Within 60 days after the insurance company's request, the insured is required to send a signed, sworn proof of loss. This document includes various details related to the loss, such as the time and cause of loss, interests of all insureds, other insurance coverage, specifications of damaged buildings, repair estimates, inventory of damaged personal property, receipts for additional living expenses, and evidence supporting specific claims.

Overall, Personal Property coverage provides insurance protection for personal belongings or possessions owned by the insured individual. It compensates for the loss or damage of various types of personal property, up to the specified coverage limit. The insured needs to provide documentation and proof of loss to support their claim.

In [48]:
Markdown(f"{response.source_nodes[0].node.get_text()}")

Sub question: What is Personal Property coverage?
Response: Personal Property coverage refers to the insurance coverage provided for personal belongings or possessions owned by the insured individual. It includes coverage for items such as furniture, clothing, electronics, jewelry, and other personal items. The coverage limit for Personal Property is typically determined based on a percentage of the limit of liability for Coverage C or a specific dollar amount, whichever is greater. 

Source: 
Document Name: HO3 Sample Policy
Category: Homeowner's Insurance
Page: 3

In [49]:
Markdown(f"{response.source_nodes[1].node.get_text()}")

Sub question: What does Personal Property coverage include?
Response: Personal Property coverage includes coverage for various types of property such as platinumplated ware, pewterware, property used primarily for business purposes, electronic apparatus and accessories, and other specified items such as money, securities, watercraft, trailers, jewelry, firearms, and silverware. 

Source: 
Document Name: HO3 Sample Policy
Category: Homeowner's Insurance
Page: 4

In [50]:
Markdown(f"{response.source_nodes[2].node.get_text()}")

Sub question: What are the limits of Personal Property coverage?
Response: The limits of Personal Property coverage are as follows:

1. Limit For Property At Other Residences: The limit of liability for personal property located at an insured's residence, other than the residence premises, is 10% of the limit of liability for Coverage C, or $1,000, whichever is greater. This limitation does not apply to personal property that is being repaired, renovated, or rebuilt and is not fit to live in or store property in, or to personal property in a newly acquired principal residence for 30 days from the time the property is moved there. (Source: HO3 Sample Policy, page 3, 2. Limit For Property At Other Residences)

2. Special Limits Of Liability: There are special limits for certain categories of personal property. These special limits do not increase the Coverage C limit of liability. The special limits are as follows:
a. $200 on money, bank notes, bullion, gold other than goldware, silver other than silverware, platinum other than platinumware, coins, medals, scrip, stored value cards, and smart cards.
b. $1,500 on securities, accounts, deeds, evidences of debt, letters of credit, notes other than bank notes, manuscripts, personal records, passports, tickets, and stamps. This limit includes the cost to research, replace, or restore the information from the lost or damaged material.
c. $1,500 on watercraft of all types, including their trailers, furnishings, equipment, and outboard engines or motors.
d. $1,500 on trailers or semitrailers not used with watercraft of all types.
e. $1,500 for loss by theft of jewelry, watches, furs, precious and semiprecious stones.
f. $2,500 for loss by theft of firearms and related equipment.
g. $2,500 for loss by theft of silverware, silver plated ware, goldware, gold plated ware, platinumware. (Source: HO3 Sample Policy, page 3-4, 3. Special Limits Of Liability)

Note: The specific page numbers and sections from the HO3 Sample Policy have been provided as sources for the answer.